# Prediction Pipeline - EfficientNetB3 Model
## Sử dụng trained model để dự đoán ảnh người dùng tải lên

Notebook này giúp bạn:
1. Load trained model từ Google Drive
2. Tải ảnh từ máy tính
3. Dự đoán class của ảnh
4. Hiển thị kết quả với confidence

## BƯỚC 1: Setup Environment

In [ ]:
import os
os.chdir('/content')

# Clone code từ repository
!rm -rf /content/python_imageNet
!git clone -b thanh https://github.com/CodeWithVu/python_imageNet.git /content/python_imageNet
%cd /content/python_imageNet
!pip install -q -e .

# Import các thư viện cần thiết
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

print("✅ TensorFlow:", tf.__version__)
print("✅ Keras:", keras.__version__)
print("✅ All libraries imported successfully!")

## BƯỚC 2: Mount Google Drive (Optional - nếu lưu model trên Drive)

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
print("✅ Google Drive mounted")

# Đường dẫn model (sửa theo vị trí của bạn)
MODELS_DIR = '/content/drive/MyDrive/DoAn_Models'  # Thay đổi theo đường dẫn của bạn

# Kiểm tra thư mục tồn tại
if not os.path.exists(MODELS_DIR):
    print(f"⚠️ Folder not found: {MODELS_DIR}")
    print("   Please create this folder on Google Drive or adjust the path")
else:
    print(f"✅ Models folder found: {MODELS_DIR}")

## BƯỚC 3: Load Trained Model

In [ ]:
# Tìm model files trong thư mục
if not os.path.exists(MODELS_DIR):
    print(f"❌ Error: Folder not found: {MODELS_DIR}")
else:
    model_files = [f for f in os.listdir(MODELS_DIR) if f.endswith('.h5')]
    
    if len(model_files) == 0:
        print(f"❌ Error: No .h5 model files found in {MODELS_DIR}")
    else:
        print(f"📁 Found {len(model_files)} model files:")
        for i, f in enumerate(model_files, 1):
            print(f"   {i}. {f}")
        
        # Lựa chọn model (chọn model mới nhất hoặc có accuracy cao nhất)
        # Ví dụ: efficientnetb3_aid_92.4.h5
        model_name = model_files[0]  # Hoặc chọn cụ thể: 'efficientnetb3_aid_92.4.h5'
        model_path = os.path.join(MODELS_DIR, model_name)
        
        print(f"\n🔄 Loading model: {model_name}")
        try:
            model = tf.keras.models.load_model(model_path)
            print(f"✅ Model loaded successfully!")
            print(f"   Input shape: {model.input_shape}")
        except Exception as e:
            print(f"❌ Error loading model: {e}")

## BƯỚC 4: Chuẩn bị Class Labels

In [ ]:
# Load class labels từ training data
from doan.data_loading import create_dataframe
from doan.data_analysis import analyze_dataset

# Path đến dataset AID (lưu ở Google Drive)
sdir = '/content/drive/MyDrive/DoAn_Python/datasets/AID'  # Sửa theo đường dẫn của bạn

# Kiểm tra dataset tồn tại
if not os.path.exists(sdir):
    print(f"❌ Error: Dataset not found at {sdir}")
    print("   Make sure you have:")
    print("   1. Uploaded AID dataset to Google Drive")
    print("   2. Path is: /content/drive/MyDrive/DoAn_Python/datasets/AID")
else:
    print("🔄 Loading dataset to get class labels...")
    try:
        df = create_dataframe(sdir)
        classes, class_count = analyze_dataset(df)
        
        print(f"✅ Found {class_count} classes")
        print(f"Classes: {classes}")
        
        # Lưu classes để sử dụng sau
        classes_list = classes
    except Exception as e:
        print(f"❌ Error loading dataset: {e}")

## BƯỚC 5: Hàm Preprocessing & Prediction

In [ ]:
def preprocess_image(image_path, img_size=(200, 200)):
    """
    Tiền xử lý ảnh để predict
    """
    # Load ảnh
    img = Image.open(image_path).convert('RGB')
    
    # Resize
    img = img.resize(img_size)
    
    # Convert to numpy array [0-255]
    img_array = np.array(img, dtype=np.float32)
    
    # Thêm batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array, img

def predict_image(model, image_path, classes_list, img_size=(200, 200)):
    """
    Dự đoán class của ảnh
    """
    # Tiền xử lý ảnh
    img_array, img_pil = preprocess_image(image_path, img_size)
    
    # Dự đoán
    predictions = model.predict(img_array, verbose=0)
    pred_probs = predictions[0]
    pred_class_idx = np.argmax(pred_probs)
    pred_class = classes_list[pred_class_idx]
    pred_confidence = pred_probs[pred_class_idx]
    
    # Top 5 predictions
    top_5_idx = np.argsort(pred_probs)[::-1][:5]
    top_5 = [(classes_list[i], pred_probs[i]) for i in top_5_idx]
    
    return {
        'class': pred_class,
        'confidence': float(pred_confidence),
        'all_probs': pred_probs,
        'top_5': top_5,
        'image': img_pil
    }

print("✅ Preprocessing & Prediction functions ready!")

## BƯỚC 6: Upload & Predict Ảnh

In [ ]:
from google.colab import files

# Upload ảnh
print("📤 Upload ảnh của bạn...")
uploaded = files.upload()

# Xử lý từng ảnh được upload
for filename, file_data in uploaded.items():
    # Lưu ảnh tạm thời
    temp_path = f'/tmp/{filename}'
    with open(temp_path, 'wb') as f:
        f.write(file_data)
    
    print(f"\n📸 Processing: {filename}")
    
    # Dự đoán
    result = predict_image(model, temp_path, classes_list)
    
    # Hiển thị kết quả
    print(f"🎯 Predicted Class: {result['class']}")
    print(f"📊 Confidence: {result['confidence']:.4f} ({result['confidence']*100:.2f}%)")
    print(f"\n🔝 Top 5 Predictions:")
    for i, (cls, conf) in enumerate(result['top_5'], 1):
        print(f"   {i}. {cls}: {conf:.4f} ({conf*100:.2f}%)")
    
    # Hiển thị ảnh
    plt.figure(figsize=(8, 6))
    plt.imshow(result['image'])
    plt.title(f"Predicted: {result['class']} ({result['confidence']*100:.2f}%)", fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Xóa file tạm
    os.remove(temp_path)

## BƯỚC 7: Batch Prediction (Optional - Dự đoán nhiều ảnh cùng lúc)

In [ ]:
def batch_predict(model, image_folder, classes_list, img_size=(200, 200)):
    """
    Dự đoán tất cả ảnh trong một thư mục
    """
    results = []
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
    
    for filename in os.listdir(image_folder):
        if filename.lower().endswith(image_extensions):
            image_path = os.path.join(image_folder, filename)
            result = predict_image(model, image_path, classes_list, img_size)
            result['filename'] = filename
            results.append(result)
    
    return results

# Ví dụ sử dụng (uncomment để chạy):
# results = batch_predict(model, '/path/to/images/folder', classes_list)
# for result in results:
#     print(f"{result['filename']}: {result['class']} ({result['confidence']:.2%})")

print("✅ Batch prediction function ready!")

## BƯỚC 8: Visualization - Biểu đồ Confidence (Optional)

In [ ]:
import matplotlib.pyplot as plt

def plot_prediction_confidence(result, num_top=10):
    """
    Vẽ biểu đồ confidence của top N classes
    """
    probs = result['all_probs']
    top_indices = np.argsort(probs)[::-1][:num_top]
    
    top_classes = [classes_list[i] for i in top_indices]
    top_probs = [probs[i] for i in top_indices]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Hiển thị ảnh
    ax1.imshow(result['image'])
    ax1.set_title(f"Predicted: {result['class']}\nConfidence: {result['confidence']:.4f}", fontsize=12, fontweight='bold')
    ax1.axis('off')
    
    # Vẽ bar chart
    colors = ['green' if i == 0 else 'steelblue' for i in range(len(top_classes))]
    ax2.barh(range(len(top_classes)), top_probs, color=colors)
    ax2.set_yticks(range(len(top_classes)))
    ax2.set_yticklabels(top_classes)
    ax2.set_xlabel('Confidence')
    ax2.set_title('Top 10 Predictions')
    ax2.invert_yaxis()
    
    # Thêm giá trị trên bars
    for i, (cls, prob) in enumerate(zip(top_classes, top_probs)):
        ax2.text(prob, i, f' {prob:.4f}', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

print("✅ Visualization function ready!")